In [58]:
import cv2
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from skimage.feature import greycomatrix,greycoprops

class Key_Frame_Extraction:
    def __init__(self,offset):
        self.numCluster = 1
        self.Cluster = []
        self.centroid = []
        self.a = offset

    def get_Frame(self,video_path):
        vidcap = cv2.VideoCapture(video_path)
        test_image_set = []
        count = 0
        frame_count = 0
        while(vidcap.isOpened):
            ret, image = vidcap.read()
            #print(ret)
            if ret:
                count +=1
                if count%2 == 0:
                    frame_count+=1
                    test_image_set.append(image)
                    #print(f"frame {frame_count}")
            else:
                    break
        vidcap.release()
        return test_image_set
        
    def CombineSimilarity(self,img,centroid):
        hist = cv2.calcHist([img],[0],None,[256],[0,256])
        hist = cv2.normalize(hist,hist).flatten()
        HisSimilarity = np.array([ cv2.compareHist(hist,x[1],cv2.HISTCMP_INTERSECT) for x in centroid])

        glcm= greycomatrix(cv2.cvtColor(img,cv2.COLOR_RGB2GRAY),[1],[0],symmetric = True,normed = True)
        GLCMSimilarity = np.array([np.sum((glcm - x[2])**2)**0.5 for x in centroid])
        return 0.7*HisSimilarity + 0.3*GLCMSimilarity
    
    def get_KeyFrame(self,frame_set):
        train_x = frame_set
        
        self.Cluster.append([train_x[0]])
        hist = cv2.calcHist([train_x[0]],[0],None,[256],[0,256])
        self.centroid.append([train_x[0],cv2.normalize(hist,hist).flatten(),
                            greycomatrix(cv2.cvtColor(train_x[0],cv2.COLOR_RGB2GRAY),[1],[0],symmetric = True,normed = True)])
        for idx,x in enumerate(train_x[1:]):
            result = self.CombineSimilarity(x,self.centroid)
            s = np.max(result)
            maxidx = np.argmax(result)
            #print(f'{s} ')
            if(s > self.a):
                #print(f'Cluster idx : {maxidx}')
                self.Cluster[maxidx].append(x)
                numFrame = len(self.Cluster[maxidx])
                hist = cv2.calcHist([self.Cluster[maxidx][numFrame//2]],[0],None,[256],[0,256])
                self.centroid[maxidx] = [self.Cluster[maxidx][numFrame//2],cv2.normalize(hist,hist).flatten(),
                                         greycomatrix(cv2.cvtColor(self.Cluster[maxidx][numFrame//2],cv2.COLOR_RGB2GRAY),[1],[0],symmetric = True,normed = True)]
            else:
                #print(f'Cluster idx : {self.numCluster}')
                self.Cluster.append([x])
                self.numCluster+=1
                hist = cv2.calcHist([x],[0],None,[256],[0,256])
                self.centroid.append([x,cv2.normalize(hist,hist).flatten(),
                            greycomatrix(cv2.cvtColor(x,cv2.COLOR_RGB2GRAY),[1],[0],symmetric = True,normed = True)])
        result= []
        for i in range(len(cl.centroid)):
            if len(cl.Cluster[i]) >= len(train_x)*0.01:
                result.append(cl.centroid[i][0])
        return result


In [59]:
cl = Key_Frame_Extraction(offset = 8)
Frame_set = cl.get_Frame("./60sec.mp4")

In [61]:
Key_Frame = cl.get_KeyFrame(Frame_set)

In [62]:
len(Key_Frame)

12

In [63]:
count = 0
for i in range(len(Key_Frame)):
    print(i)
    cv2.imwrite('60_8_0.01_' + str(count) + '.jpg',Key_Frame[i])
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
